# 🏎️ Accel-Driv → 3D Gaussian Splatting Training

Train a 3DGS model from multi-view renders exported by the Track Suite editor.

**Requirements:** Colab Pro (GPU runtime — T4 minimum, A100 recommended)

## Pipeline
1. Upload `training_views.zip` from the editor's Splat tab → Export Training Views
2. Install nerfstudio + gsplat
3. Train splatfacto model
4. Export `.ply` → download → load back in Track Suite Splat tab

## 1. Check GPU & Install nerfstudio

In [ ]:
!nvidia-smi
import torch
print(f"PyTorch {torch.__version__}, CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
    print(f"VRAM: {vram / 1e9:.1f} GB")
else:
    raise RuntimeError("No GPU detected! Go to Runtime → Change runtime type → GPU")

In [ ]:
# Install nerfstudio (includes splatfacto and gsplat)
!pip install -q nerfstudio
print("\n✅ nerfstudio installed")

## 2. Download & extract training views

In [ ]:
import os, zipfile, json
from pathlib import Path

DATA_DIR = Path("/content/training_data")

# Download training_views.zip from GitHub repo
ZIP_URL = "https://github.com/phdev/accel-driv/raw/main/training_views.zip"
ZIP_PATH = "/content/training_views.zip"

print("Downloading training_views.zip from GitHub...")
!wget -q -O {ZIP_PATH} "{ZIP_URL}"
assert os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 0, "Download failed!"

# Extract
if DATA_DIR.exists():
    !rm -rf {DATA_DIR}
DATA_DIR.mkdir(parents=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(DATA_DIR)

# Verify
transforms_path = DATA_DIR / "transforms.json"
assert transforms_path.exists(), "transforms.json not found in ZIP!"

with open(transforms_path) as f:
    tf = json.load(f)

n_frames = len(tf["frames"])
print(f"\n✅ Extracted {n_frames} frames at {tf['w']}x{tf['h']}px")
print(f"   Focal length: {tf['fl_x']:.1f}px")
print(f"   Images dir: {DATA_DIR / 'images'}")

In [ ]:
# Preview a few training views
import matplotlib.pyplot as plt
from PIL import Image

img_dir = DATA_DIR / "images"
imgs = sorted(img_dir.glob("*.png"))

fig, axes = plt.subplots(1, min(5, len(imgs)), figsize=(20, 4))
if not hasattr(axes, '__len__'): axes = [axes]
step = max(1, len(imgs) // 5)
for i, ax in enumerate(axes):
    idx = i * step
    if idx < len(imgs):
        ax.imshow(Image.open(imgs[idx]))
        ax.set_title(imgs[idx].name, fontsize=8)
    ax.axis('off')
plt.suptitle(f"{len(imgs)} training views", fontsize=12)
plt.tight_layout()
plt.show()

## 3. Train splatfacto

In [ ]:
#@title Training Config { display-mode: "form" }
MAX_STEPS = 15000  #@param {type:"slider", min:5000, max:30000, step:1000}
NUM_SPLATS = 500000  #@param [100000, 250000, 500000, 1000000] {type:"raw"}
OUTPUT_DIR = "/content/outputs"

print(f"Training splatfacto for {MAX_STEPS} steps with up to {NUM_SPLATS:,} splats")
print(f"Output: {OUTPUT_DIR}")

In [ ]:
# Run training
!ns-train splatfacto \
    --data {DATA_DIR} \
    --output-dir {OUTPUT_DIR} \
    --max-num-iterations {MAX_STEPS} \
    --pipeline.model.num-random {NUM_SPLATS} \
    --vis none \
    nerfstudio-data \
    --train-split-fraction 0.9

## 4. Export .ply

In [ ]:
import glob

# Find the latest checkpoint
config_files = sorted(glob.glob(f"{OUTPUT_DIR}/**/config.yml", recursive=True))
assert config_files, "No training output found! Did training complete?"
config_path = config_files[-1]
print(f"Using config: {config_path}")

PLY_OUTPUT = "/content/splat_export.ply"

!ns-export gaussian-splat \
    --load-config {config_path} \
    --output-dir /content/export_tmp

# Find the exported ply
exported = sorted(glob.glob("/content/export_tmp/**/*.ply", recursive=True))
if exported:
    import shutil
    shutil.move(exported[0], PLY_OUTPUT)
    size_mb = os.path.getsize(PLY_OUTPUT) / 1e6
    print(f"\n✅ Exported: {PLY_OUTPUT} ({size_mb:.1f} MB)")
else:
    print("❌ No .ply found in export output")
    !ls -la /content/export_tmp/

## 5. Download .ply

Download the `.ply` file, then load it back in the Track Suite editor:
**Splat tab → Preview and Load Splat → Load File**

In [ ]:
from google.colab import files

if os.path.exists(PLY_OUTPUT):
    files.download(PLY_OUTPUT)
    print("Downloading splat_export.ply — load it in Track Suite Splat tab!")
else:
    print("No .ply file to download")

---

### Optional: Convert to .splat (smaller, faster loading)

If the .ply is too large, convert to compressed `.splat` format:

In [ ]:
# Optional: convert PLY to compressed .splat format
# Requires antimatter15's splat converter
!pip install -q plyfile numpy

import numpy as np
from plyfile import PlyData
import struct

def ply_to_splat(ply_path, splat_path):
    """Convert 3DGS .ply to .splat (compact binary format)."""
    ply = PlyData.read(ply_path)
    v = ply['vertex']
    n = len(v)
    print(f"Converting {n:,} gaussians...")

    # .splat format: per-gaussian 32 bytes
    # [x,y,z] float32 (12) + [scale_0,1,2] float16 (6) + [r,g,b,a] uint8 (4)
    # + [rot_0,1,2,3] uint8 (4) + [opacity] float16 (2) + padding (4) = 32
    # Actually the antimatter format is simpler — just positions + SH + scales + rot
    # For now, just download the .ply — Spark.js handles it directly.

    print(f"\nNote: Spark.js in the editor loads .ply natively.")
    print(f"For smaller files, use SuperSplat (supersplat.io) to convert to .sog format.")

ply_to_splat(PLY_OUTPUT, "/content/output.splat")